**In case** there is issue with reloading, run the following to delete compiled Python cache.

This is **typically** not needed.
Often, restarting the notebook works better.

In [1]:
!cd ..; cd arguments; rm -rf __pycache__
!cd ..; cd constants; rm -rf __pycache__
!cd ..; cd data; rm -rf __pycache__
!cd ..; cd logger; rm -rf __pycache__
!cd ..; cd models; rm -rf __pycache__
!cd ..; cd utils; rm -rf __pycache__

In [1]:
%load_ext autoreload
%autoreload 2

# Entrace Notebook for Forex Trading

This is for testing only. For batched runs a main file is still required.

Keep autoreload for faster Python module reloading.

## Imports

Import Python default packges first, then those requiring `pip`, finally modules in the code structure. Keep alphabetical order.

In [1]:
import sys
sys.path.append('..')

import os

import pandas as pd
import tqdm

from arguments import *
from constants import *
from data import *
import models as supported_models
from training_tools import *
from utils import *

## Preparations

* Setup interface with Google Cloud bucket.
* Pretend to have command line argument.
* Prepare data
* Define models

In [2]:
storage = GCStorage.get_CloudFS(project_name=PROJECT_NAME,
                                bucket_name=GC_BUCKET,
                                credential_path=CREDENTIAL_PATH)

Unique instance for GCStorage has been created


In [3]:
all_args = parse_from_string('--exp_name=jupyter --device=cpu --num_candles=20 --num_workers=1 '
                             '--fast_debug=True --candle_interval=30 --log_level=4')

Starting new experiment at 2020-05-07 00:43:21
User: jingbo
Host: deep3.stanford.edu
{'misc_args': {'exp_name': 'jingbo_jupyter_5', 'log_level': 4, 'fast_debug': True, 'save_dir': 'experiments/2020-05-07/jingbo_jupyter_5', 'log_file': 'experiments/2020-05-07/jingbo_jupyter_5/run_log.txt'}, 'data_args': {'candle_interval': 30, 'num_candles': 20, 'num_iterval_ahead': 4, 'currency_pair': 'USDCAD', 'num_workers': 1}, 'train_args': {'device': 'cpu', 'disp_steps': 1, 'max_epochs': 100, 'batch_size': 256, 'learning_rate': 0.0001, 'weight_decay': 0.9, 'clipping_value': 1.0}, 'model_args': {'model_type': 'DummyModel', 'emb_size': 32, 'hidden_size': 64, 'num_layers': 3}}
Step 0/False Texts
	[setup/command_line: /deep/group/packages/lilsnake3/envs/eeg/lib/python3.6/site-packages/ipykernel_launcher.py --exp_name=jupyter --device=cpu --num_candles=20 --num_workers=1 --fast_debug=True --candle_interval=30 --log_level=4]
	[setup/arguments: {   'data_args': {   'candle_interval': 30,
                 

In [4]:
logger = all_args.misc_args.logger
device = all_args.train_args.device

In [5]:
train_loader, valid_loader = get_dataloaders(all_args)

Unique instance for DataAPI has been created


In [6]:
model_init_func = supported_models.__dict__[all_args.model_args.model_type]
model = model_init_func(all_args.model_args) # TODO: JBY. Enable Dataparallel, if needed.
model = model.to(device)

## Define Training Tools

In [7]:
loss_func = get_loss_func()# TODO (JBY): Replace with proper evaluator
optimizer = Optimizer(all_args, model.parameters(), len(train_loader.dataset))

## Train (Deep Learning)

At the moment we only have a validation set, a test set has not been created yet.

In [ ]:
for epoch in range(all_args.train_args.max_epochs):
    for batch in tqdm(train_loader):
        
        optimizer.zero_grad()
        time_series, ground_truth = batch

        pred = model(time_series.float())
        
        loss = loss_func(pred, ground_truth.float())
        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly

    print(f'Epoch: {epoch + 1}/{all_args.train_args.max_epochs}')
    print(f'Loss: {loss.item()}')




  0%|          | 0/23 [00:00<?, ?it/s]


  4%|▍         | 1/23 [00:37<13:48, 37.64s/it]


  9%|▊         | 2/23 [00:55<11:03, 31.57s/it]


 13%|█▎        | 3/23 [01:12<09:07, 27.39s/it]


 17%|█▋        | 4/23 [01:30<07:46, 24.56s/it]


 22%|██▏       | 5/23 [01:47<06:41, 22.32s/it]


 26%|██▌       | 6/23 [02:04<05:51, 20.69s/it]


 30%|███       | 7/23 [02:22<05:15, 19.73s/it]


 35%|███▍      | 8/23 [02:38<04:41, 18.80s/it]


 39%|███▉      | 9/23 [02:56<04:17, 18.39s/it]


 43%|████▎     | 10/23 [03:13<03:55, 18.12s/it]